In [62]:
import os, shutil, pickle, shelve
from Bio import SeqIO
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import load_model, Model
from keras.utils import plot_model
from keras.models import Model, Sequential
from keras.models import Sequential
from keras.layers import Dense, Dropout, MaxPooling1D, Flatten, Conv1D, LSTM, CuDNNLSTM, Bidirectional
from keras.optimizers import RMSprop, Adam
from keras.initializers import glorot_normal

In [63]:
# sample data from:
data_dir = "/home/ubuntu/group_volume/team_neural_network/data/input/3.24_species_only"
output_folder_path = "../../../../temp/buffers/"

## 1. Training and validation

In [64]:
def pad_for_detector(input_x, kernel_size):
    """ Pad the input matrix such that the (i, k) entry of the output 
        matrix is the score of motif detector k aligned to position i.
    input_x has shape = (N, n, 4)
    kernel_size has shape m
    output has shape = (N, n + 2m - 2, 4)
    """
    N, n, C = input_x.shape
    pad_value, num_pad = 0.25, kernel_size - 1
    pad_matrix = np.full((N, num_pad, C), pad_value)
    return np.concatenate((pad_matrix, input_x, pad_matrix), axis=1)

def one_hot_encoding(input_folder_path, output_file_path, 
                     max_file_num=10000):
    """
    Given the data in INPUT_FOLDER_PATH, encode them and save
    as a buffer called OUTPUT_FILE_PATH.
    
    Note: INPUT_FOLDER_PATH is a directory while OUTPUT_FILE_PATH
          is a file.
    """
    import os
    # Use the following dictionary to perform the transformation
    base_pairs = {'A': [1, 0, 0, 0], 
                  'C': [0, 1, 0, 0],
                  'G': [0, 0, 1, 0],
                  'T': [0, 0, 0, 1],
                  'a': [1, 0, 0, 0],
                  'c': [0, 1, 0, 0],
                  'g': [0, 0, 1, 0],
                  't': [0, 0, 0, 1],
                  'n': [0, 0, 0, 0],
                  'N': [0, 0, 0, 0]}

    file_num_limit = max_file_num    # The maximum number of files to be decoded
    file_count = 0

    # Iterate through every file
    all_regions = []
    for file in os.listdir(input_folder_path):
        # When the number of file decoded has reached the limit, stop
        if file_count < file_num_limit:
            #print(input_folder_path + file)
            data = list(SeqIO.parse(input_folder_path + file,"fasta"))
            for n in range(0, len(data)):
                # Extract the header information
                header = data[n].description.split('|')
                descr = data[n].description
                regionID = header[0]
                expressed = header[1]
                speciesID = header[2]
                strand = header[3]
                # Complement all sequences in the negative DNA strand
    #             if strand == '-':
    #                 # Using the syntax [e for e in base_pairs[n]] to create a new pointer for each position
    #                 one_hot.append([descr, expressed, speciesID, [[e for e in base_pairs[n]] for n in data[n].seq.complement()]])
    #             else:
                all_regions.append([descr, expressed, speciesID, [[e for e in base_pairs[n]] for n in data[n].seq]])
            file_count += 1

    with open(output_file_path, mode="wb") as output:
        print("save to {}".format(output_file_path))
        pickle.dump(all_regions, output)
    return all_regions

def get_training_data(input_data, output_folder_path,
                      max_len, train_x_name, train_y_name):
    """ 
    Convert INPUT_DATA to ready-to-be-fed training data and 
        corresponding labels.
    Save them to OUTPUT_FOLDER_PATH with name TRAIN_X_NAME and
        TRAIN_Y_NAME.
    INPUT_DATA is directly generated by the function one_hot_encoding. 
    """
    train_x, train_y = [], []
    for region in input_data:
        y, x = int(region[1]), region[3]
        x = curtail(x, max_len)  # Curtail
        x = np.array(x).flatten() # Flatten
        x = x.reshape((1000, 4)) # Reshape
        train_x.append(x)
        train_y.append(y)

    train_x, train_y = np.array(train_x), np.array(train_y)

    print(train_x.shape, train_y.shape)

    with open(os.path.join(output_folder_path, train_x_name), mode="wb") as output:
        print("save to {}".format(os.path.join(output_folder_path, train_x_name)))
        pickle.dump(train_x, output)

    with open(os.path.join(output_folder_path, train_y_name), mode="wb") as output:
        print("save to {}".format(os.path.join(output_folder_path, train_y_name)))
        pickle.dump(train_y, output)
    return train_x, train_y

def dianostic_plots(train_acc, train_loss, val_train_acc, val_loss):
    """  Plot dianostic plots of a model:
    Plot 1: Traning loss & validation loss against epochs
    Plot 2: Training acc & validation acc against epochs
    """
    epochs = range(1, len(train_acc) + 1)

    plt.plot(epochs, train_acc, '-', label='Training train_accuracy')
    plt.plot(epochs, val_train_acc, '-', label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('epoches')
    plt.legend()

    plt.figure()
    plt.plot(epochs, loss, '-', label='Training Loss')
    plt.plot(epochs, val_loss, '-', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('epoches')
    plt.legend()
    
    plt.show()

In [65]:
!rm -r /home/ubuntu/data/temp/train
!mkdir /home/ubuntu/data/temp/train
!rm -r /home/ubuntu/data/temp/val
!mkdir /home/ubuntu/data/temp/val

In [66]:
# Sample training and validation data
# Make sure that they don't have intersection.
all_data_lst = np.array(os.listdir(data_dir))
n = len(all_data_lst)
train_files = all_data_lst[:500]
num_val = 100
val_indices = np.random.choice(np.arange(500, n), num_val, replace = False)
val_files = all_data_lst[val_indices]

train_dest = '/home/ubuntu/data/temp/train/'
for file in train_files:
    shutil.copy(os.path.join(data_dir, file),
                          train_dest)
print('copied training samples to {}'.format(train_dest))

val_dest = '/home/ubuntu/data/temp/val/'
for file in val_files:
    shutil.copy(os.path.join(data_dir, file),
                          val_dest)
print('copied validation samples to {}'.format(val_dest))

# Preprocess train and val data so that they are ready to be fed to models
train_output_path = os.path.join(output_folder_path, 'train.data')
val_output_path = os.path.join(output_folder_path, 'val.data')

train_regions = one_hot_encoding(train_dest, train_output_path)
val_regions = one_hot_encoding(val_dest, val_output_path)
train_x, train_y = get_training_data(train_regions, output_folder_path,
                                   max_len = 1000, 
                                   train_x_name = 'train_x.data', 
                                   train_y_name = 'train_y.data')
val_x, val_y = get_training_data(val_regions, output_folder_path,
                                   max_len = 1000, 
                                   train_x_name = 'val_x.data', 
                                   train_y_name = 'val_y.data')
# Pad for motif detectors
train_x, val_x = pad_for_detector(train_x, 15), pad_for_detector(val_x, 15)

copied training samples to /home/ubuntu/data/temp/train/
copied validation samples to /home/ubuntu/data/temp/val/
save to ../../../../temp/buffers/train.data
save to ../../../../temp/buffers/val.data
(12000, 1000, 4) (12000,)
save to ../../../../temp/buffers/train_x.data
save to ../../../../temp/buffers/train_y.data
(2400, 1000, 4) (2400,)
save to ../../../../temp/buffers/val_x.data
save to ../../../../temp/buffers/val_y.data


In [67]:
def get_hybrid(opt):
    """  Return a hybrid network given a optimizer
    """
    model = Sequential()
    model.add(Conv1D(filters = 30, 
                     kernel_size = 15, 
                     padding = 'valid',
                     data_format = 'channels_last',
                     activation = 'relu'))
    model.add(MaxPooling1D(pool_size = 5, strides = 5))
    model.add(Dropout(0.6))
    model.add(Bidirectional(LSTM(10)))
    model.add(Dropout(0.7))
    model.add(Dense(20, activation = 'relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['acc'])
    return model

def train(model, train_x, train_y, val_data, config = {'epochs': 35, 'batch_size': 256}):
    """  Train model for a given config, training data, and validation data
    """
    epochs, batch_size = config['epochs'], config['batch_size']
    return model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, validation_data=val_data)

In [68]:
# Start training
# Set up some configurations
optimizers = {'adam': Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1e-2, amsgrad=False),
              'rmsprop': RMSprop(lr=1e-2, rho=0.9, epsilon=None, decay=1e-2)}
config = {'epochs': 35, 'batch_size': 256}
opt = optimizers['rmsprop']
model = get_hybrid(opt)
history = train(model, train_x, train_y, (val_x, val_y), config)

Train on 12000 samples, validate on 2400 samples
Epoch 1/35
12000/12000 [==============================] - 40s 3ms/step - loss: 0.6901 - acc: 0.5488 - val_loss: 0.6894 - val_acc: 0.5400
Epoch 2/35
12000/12000 [==============================] - 35s 3ms/step - loss: 0.6875 - acc: 0.5522 - val_loss: 0.6680 - val_acc: 0.5400
Epoch 3/35
12000/12000 [==============================] - 35s 3ms/step - loss: 0.6845 - acc: 0.5513 - val_loss: 0.6627 - val_acc: 0.5400
Epoch 4/35
12000/12000 [==============================] - 35s 3ms/step - loss: 0.6846 - acc: 0.5518 - val_loss: 0.6631 - val_acc: 0.5400
Epoch 5/35
12000/12000 [==============================] - 35s 3ms/step - loss: 0.6843 - acc: 0.5505 - val_loss: 0.6578 - val_acc: 0.5400
Epoch 6/35
12000/12000 [==============================] - 35s 3ms/step - loss: 0.6822 - acc: 0.5508 - val_loss: 0.6558 - val_acc: 0.5808
Epoch 7/35
12000/12000 [==============================] - 35s 3ms/step - loss: 0.6790 - acc: 0.5581 - val_loss: 0.6519 - val_acc:

KeyboardInterrupt: 

In [ ]:
model.summary()

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

dianostic_plots(acc, loss, val_acc, val_loss)

In [ ]:
# Save the model for later analysis
save_model_path = '/home/ubuntu/data/team_neural_network/code/models'
model_name = 'hybrid_net2.h5'
model.save(os.path.join(save_model_path, model_name))

## 2. Retrieve motif from kernel

In [184]:
def get_activated_subseq(activations, test_seq, m):
    #################################################################
    # Extract those in the TEST_SEQUENCE that has least one position
    #     having positive activation.
    #################################################################
    N, L, C = activations.shape
    activated_subseq = {}
    for i in range(C): # for each filter
        activated_subseq[i] = []
        activation = activations[:, :, i]
        
        # candidate and potential_start has a shape (N, )
        candidate, potential_start = np.max(activation, axis=1), np.argmax(activation, axis=1)
        
        # activated_indices should have a shape (K, ), where K = # of positive activations
        activated_indices = [i for i in range(N) if candidate[i] > 0]
        K = len(activated_indices)
        
        # activated_seq should have a shape(K, n + 2m - 2, 4)
        activated_seq, ends = test_seq[activated_indices, :, :], potential_start[activated_indices]
        starts = ends - m + 1
        for k in range(K):
            start, end = starts[k], ends[k]
            activated_subseq[i].append(activated_seq[k, start:(end+1), :])
    return activated_subseq

In [50]:
model = load_model('../../models/hybrid_net.h5')

filters = model.layers[0].get_weights()[0]
bias = model.layers[0].get_weights()[1]

In [180]:
N, n, m = 100, 1000, 15
test_seq = np.random.randint(2, size = N*n*4).reshape((N, n, 4))
padded_test_seq = pad_for_detector(test_seq, m)
padded_test_seq.shape

(100, 1028, 4)

In [181]:
layer_name = model.layers[0].get_config()['name']
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict(test_seq)

In [182]:
intermediate_output.shape

(100, 986, 30)

In [185]:
activated_subseq = get_activated_subseq(intermediate_output, test_seq, m)

In [188]:
activated_subseq[0][0]

[array([1, 0, 1, 1]),
 array([1, 0, 0, 0]),
 array([1, 0, 1, 1]),
 array([1, 0, 1, 1]),
 array([1, 0, 0, 1]),
 array([0, 1, 1, 0]),
 array([0, 1, 1, 1]),
 array([1, 1, 0, 0]),
 array([1, 1, 0, 1]),
 array([1, 0, 0, 0]),
 array([0, 0, 0, 1]),
 array([0, 0, 1, 0]),
 array([1, 1, 1, 1]),
 array([1, 0, 0, 1]),
 array([1, 0, 1, 0])]

In [ ]:
def convert_to_car(one_hot_code):
    char_to_code = {'A': np.array([1, 0, 0, 0]),
                    'C': np.array([0, 1, 0, 0]),
                    'G': np.array([0, 0, 1, 0]),
                    'T': np.array([0, 0, 0, 1]),
                    'N': np.array([0.25, 0.25, 0.25, 0.25])}
    for char in ['A', 'C', 'G', 'T', 'N']:
        if sum(char_to_code[char] - one_hot_code) == 0:
            return char

def get_pwm(activated_subseq):
    pwm = {}
    for i in range(m):
        subseqs = activated_subseq[i]
        counter = {char: 0 for char in ['A', 'C', 'G', 'T']}
        for j in range(len(subseqs)):
            subseq = list(subseqs[j])
            for code in subseq:
                c = convert_to_car(code)
                counter[c] += 1
        pwm[i] = counter.copy()
    return pwm